In [7]:
import sqlite3
import random
import re

In [8]:
# DATABASE SETUP


conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    city TEXT,
    occupation TEXT,
    salary INTEGER
)
''')

names = ["Alice", "Bob", "Charlie", "Dante", "Eva", "Mia", "Liam", "Noah", "Sophia", "Ava"]
cities = ["New York", "Paris", "Los Angeles", "London", "Tokyo", "Berlin", "Istanbul"]
occupations = ["Engineer", "Doctor", "Designer", "Teacher", "Hacker", "Analyst", "Researcher"]

for _ in range(150):
    cursor.execute(
        "INSERT INTO users (name, age, city, occupation, salary) VALUES (?, ?, ?, ?, ?)",
        (
            random.choice(names),
            random.randint(18, 65),
            random.choice(cities),
            random.choice(occupations),
            random.randint(30000, 150000)
        )
    )
conn.commit()

In [9]:
# CITY NORMALIZATION
CITY_SYNONYMS = {
    "nyc": "New York",
    "la": "Los Angeles",
    "tok": "Tokyo"
}

def normalize_city(city):
    city = city.strip().title()
    return CITY_SYNONYMS.get(city.lower(), city)

In [10]:
def nlq_to_sql(text: str) -> str:
    original = text
    t = text.lower()
    conditions = []
    select = "*"
    order = ""
    limit = ""

    # ----------------------------------------
    # SELECT detection
    # ----------------------------------------
    if any(phrase in t for phrase in ["average age", "avg age", "mean age"]):
        select = "AVG(age)"
    elif any(phrase in t for phrase in ["count", "how many", "number of"]):
        select = "COUNT(*)"
    elif any(phrase in t for phrase in ["names", "name of", "who are"]):
        select = "name"
    elif "salary" in t and ("average" in t or "avg" in t or "mean" in t):
        select = "AVG(salary)"
    elif ("salary" in t or "salaries" in t) and ("sum" in t or "total" in t):
        select = "SUM(salary)"
    elif "salaries" in t or ("salary" in t and select == "*"):
        select = "salary"

    # ----------------------------------------
    # OCCUPATIONS
    # ----------------------------------------
    occ_pattern = r"\b(engineer|doctor|designer|teacher|hacker|analyst|researcher)s?\b"
    occ_matches = re.findall(occ_pattern, t)

    if occ_matches:
        occupations = sorted({o.capitalize() for o in set(occ_matches)})
        if len(occupations) == 1:
            conditions.append(f"occupation = '{occupations[0]}'")
        else:
            joined = ", ".join(f"'{o}'" for o in occupations)
            conditions.append(f"occupation IN ({joined})")

        # Remove occupation words to avoid interfering with city parsing
        for occ in set(occ_matches):
            t = re.sub(rf"\b{occ}s?\b", "", t)

    # ----------------------------------------
    # CITY PARSING – now supports multiple cities with "and"
    # ----------------------------------------
    # First, remove known filter keywords so they don't interfere
    filter_keywords = [
        "older", "younger", "between", "salary", "with", "sort", "order",
        "top", "limit", "average", "avg", "count", "how many", "name", "names"
    ]
    temp_t = t
    for kw in filter_keywords:
        temp_t = re.sub(rf"\b{kw}\b", " _FILTER_ ", temp_t)

    # Extract everything after "from"
    from_part = re.search(r"\bfrom\s+(.+)", temp_t)
    cities = []

    if from_part:
        city_text = from_part.group(1)

        # Split on "and" or commas, but respect boundaries
        candidates = re.split(r'\s+and\s+|\s*,\s*', city_text)
        for cand in candidates:
            cand = cand.strip()
            if cand and not cand.startswith("_filter_"):
                # Extract clean city name (stop before next filter keyword)
                clean = re.sub(r"_filter_.*", "", cand).strip()
                if clean:
                    norm_city = normalize_city(clean)
                    if norm_city:
                        cities.append(norm_city)

    if cities:
        if len(cities) == 1:
            conditions.append(f"city = '{cities[0]}'")
        else:
            joined = ", ".join(f"'{c}'" for c in cities)
            conditions.append(f"city IN ({joined})")

    # ----------------------------------------
    # AGE filters
    # ----------------------------------------
    if match := re.search(r"older than (\d+)", t):
        conditions.append(f"age > {match.group(1)}")
    if match := re.search(r"younger than (\d+)", t):
        conditions.append(f"age < {match.group(1)}")
    if match := re.search(r"between (\d+) and (\d+)", t):
        a, b = match.groups()
        conditions.append(f"age BETWEEN {min(int(a), int(b))} AND {max(int(a), int(b))}")

    # ----------------------------------------
    # SALARY filters
    # ----------------------------------------
    if match := re.search(r"salary over (\d+)", t):
        conditions.append(f"salary > {match.group(1)}")
    if match := re.search(r"salary (?:under|below) (\d+)", t):
        conditions.append(f"salary < {match.group(1)}")
    if match := re.search(r"salary (?:above|greater than) (\d+)", t):
        conditions.append(f"salary > {match.group(1)}")

    # ----------------------------------------
    # ORDER / SORT
    # ----------------------------------------
    if re.search(r"\b(sort|order)\b.+age", t):
        order = " ORDER BY age"
        if re.search(r"\bdescending|desc|highest|oldest", t):
            order += " DESC"
        else:
            order += " ASC"  # default to ASC if not specified

    if re.search(r"\b(sort|order)\b.+salary", t):
        order = " ORDER BY salary"
        if re.search(r"\bdescending|desc|highest", t):
            order += " DESC"

    # ----------------------------------------
    # LIMIT (“top N”, “first N”, etc.)
    # ----------------------------------------
    if match := re.search(r"\b(top|first|limit)\s+(\d+)", t):
        limit = f" LIMIT {match.group(2)}"

    # ----------------------------------------
    # FINAL SQL
    # ----------------------------------------
    sql = f"SELECT {select} FROM users"
    if conditions:
        sql += " WHERE " + " AND ".join(conditions)
    sql += order + limit

    return sql.strip()

In [11]:
while True:
    q = input("Query: ")
    if q.lower().strip() == "exit":
        break
    elif q == "":
        continue

    sql = nlq_to_sql(q)
    print(f"\nNLQ: {q}")
    print(f"SQL: {sql}")

    cursor.execute(sql)
    result = cursor.fetchall()
    print("Result:", result, "\n")


NLQ: list users
SQL: SELECT * FROM users
Result: [(1, 'Mia', 23, 'Los Angeles', 'Engineer', 119046), (2, 'Mia', 44, 'New York', 'Engineer', 140029), (3, 'Eva', 26, 'New York', 'Doctor', 107244), (4, 'Dante', 26, 'New York', 'Hacker', 45349), (5, 'Ava', 42, 'Los Angeles', 'Doctor', 96108), (6, 'Liam', 40, 'Istanbul', 'Researcher', 46014), (7, 'Bob', 63, 'Berlin', 'Analyst', 85439), (8, 'Eva', 40, 'Tokyo', 'Analyst', 39449), (9, 'Alice', 61, 'Istanbul', 'Hacker', 136931), (10, 'Liam', 38, 'Tokyo', 'Researcher', 114666), (11, 'Alice', 53, 'Paris', 'Teacher', 48454), (12, 'Bob', 49, 'Istanbul', 'Doctor', 62052), (13, 'Mia', 47, 'New York', 'Hacker', 50714), (14, 'Ava', 34, 'London', 'Doctor', 51573), (15, 'Eva', 30, 'New York', 'Designer', 66587), (16, 'Dante', 60, 'Tokyo', 'Doctor', 66427), (17, 'Noah', 58, 'Berlin', 'Doctor', 98845), (18, 'Mia', 24, 'London', 'Designer', 144175), (19, 'Noah', 53, 'London', 'Doctor', 87932), (20, 'Mia', 46, 'Tokyo', 'Engineer', 114949), (21, 'Noah', 57, 

In [12]:
conn.close()